In [1]:
import pandas as pd
import numpy as np
import requests
import time
from googleapiclient.discovery import build
from IPython.display import JSON

In [2]:
api_key = ""

# get credentials and create an API client
youtube = build("youtube", "v3", developerKey=api_key)

In [3]:
channel_ids = ["UCG8rbF3g2AMX70yOd8vqIZg", # Logan Paul
               "UCLhvu_-py2eiC_yD17WLLNg", # Learn & Play With Zack
               "UCAgx4HcQIYn9lM0rhtIuH9w", # HZHtube Fun Kids
              ]

In [4]:
request = youtube.channels().list(
        part="snippet,contentDetails,statistics,topicDetails,status",
        id=','.join(channel_ids)
)
response = request.execute()

In [5]:
JSON(response)

<IPython.core.display.JSON object>

In [6]:
def get_channel_data(youtube, channel_ids):
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics,topicDetails,status",
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    all_data = []
    
    for i in range(len(channel_ids)):
    
        channel = response['items'][i]

        channel_data = {
            'channel_id': channel['id'],
            'channel_name': channel['snippet']['title'],
            'channel_desc': channel['snippet']['description'],
            'channel_start_date': channel['snippet']['publishedAt'],
            'total_views': channel['statistics']['viewCount'],
            'total_subscribers': channel['statistics']['subscriberCount'],
            'total_videos_posted': channel['statistics']['videoCount']
        }
        
        try:
            channel_data['channel_topic'] = channel['topicDetails']['topicCategories']
        except:
            channel_data['channel_topic'] = None
        
        all_data.append(channel_data)
        
    return pd.DataFrame(all_data)
        

In [7]:
get_channel_data(youtube, channel_ids)

,channel_id,channel_name,channel_desc,channel_start_date,total_views,total_subscribers,total_videos_posted,channel_topic
0,UCG8rbF3g2AMX70yOd8vqIZg,Logan Paul,27 year old in Puerto Rico living life to the ...,2015-08-29T03:28:33Z,5944653195,23600000,715,"[https://en.wikipedia.org/wiki/Entertainment, ..."
1,UCLhvu_-py2eiC_yD17WLLNg,Heidi Zidane and Zack,Welcome to Heidi Zidane and Zack Youtube Chann...,2018-03-23T14:32:26Z,200246680,1340000,342,"[https://en.wikipedia.org/wiki/Hobby, https://..."
2,UCAgx4HcQIYn9lM0rhtIuH9w,Heidi and Zidane HZHtube,,2015-11-24T19:25:04Z,3607762032,12400000,686,"[https://en.wikipedia.org/wiki/Hobby, https://..."


### Most Popular Videos

In [8]:
def get_most_popular_videos(youtube):
    
    request = youtube.videos().list(
        part="snippet,contentDetails,status,statistics,topicDetails",
        chart="mostPopular",
        regionCode="US",
        maxResults = 50
    )
    response = request.execute()

    all_data = []

    for i in range(50):
        video = response['items'][i]

        video_data = {
            'video_id': video['id'],
            'channel_id': video['snippet']['channelId'],
            'video_posting_date': video['snippet']['publishedAt'],
            'video_title': video['snippet']['title'],
            'video_desc': video['snippet']['description'],
            'video_length': video['contentDetails']['duration'],
            'made_for_kids': video['status']['madeForKids'],
            'video_views': video['statistics']['viewCount'],
            'video_comments': video['statistics']['commentCount']
        }

        try:
            video_data['video_topic'] = video['topicDetails']['topicCategories']
            video_data['video_likes'] = video['statistics']['likeCount']
        except:
            video_data['video_topic'] = None
            video_data['video_likes'] = None
        all_data.append(video_data)
    
    return pd.DataFrame(all_data)

In [9]:
popular_videos = get_most_popular_videos(youtube)
popular_videos.head()

,video_id,channel_id,video_posting_date,video_title,video_desc,video_length,made_for_kids,video_views,video_comments,video_topic,video_likes
0,vT58GKZpXNs,UC2mP7il3YV7TxM_3m6U0bwA,2023-01-31T21:00:21Z,Ranking Your INSANE Tiktoks,Try Rocket Money for free! https://rocketmoney...,PT29M7S,False,1136645,4340,"[https://en.wikipedia.org/wiki/Humour, https:/...",100830
1,5jxyH6GYLsk,UC4PQqjGczpgmqbpicKjkwvw,2023-01-31T20:00:07Z,Coulda Been Records LA Auditions hosted by Druski,Here’s your chance to join me live on stage fo...,PT34M48S,False,471773,1971,"[https://en.wikipedia.org/wiki/Entertainment, ...",24274
2,qMXESlny4-I,UCDE5Ezmxq1bNVak4lmkpCMw,2023-01-31T14:59:47Z,"Falling In Reverse - ""Watch The World Burn""","""Watch The World Burn"" by @FallingInReverse \n...",PT3M24S,False,995247,7981,"[https://en.wikipedia.org/wiki/Music, https://...",90351
3,6ayJjfyRRkc,UCR_J_SntqJh5eXw66d5hJxA,2023-01-31T23:53:05Z,I Challenged iShowSpeed to a 1v1 Soccer Game!,I built @IShowSpeed a custom Cristiano Ronaldo...,PT8M37S,False,1170349,2494,[https://en.wikipedia.org/wiki/Association_foo...,33791
4,JT-Kh5iknkE,UCnmGIkw-KdI0W5siakKPKog,2023-01-31T15:00:12Z,I Excavated a Fossil Site,digging for dinosaurs is fun\nread my full sto...,PT9M51S,False,2076786,4460,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,100127
